In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
from collections import defaultdict

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [5]:
def getAprByShards():
    count = defaultdict(int)
    apr_sum = defaultdict(int)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            apr = float(i['lifetime']['apr'])
            for s in i['metrics']['by-shard']:
                shard = s['shard-id']
                count[shard] += 1
                apr_sum[shard] += apr
    apr_avg = dict()
    for k,v in apr_sum.items():
        apr_avg[k] = v/count[k]
    return apr_avg

In [6]:
def R14_test():
    print("Test-R14: Shard fairness: rate of earning on shards is similar if the block time are same")
    num = int(input("enter the number of blocks you want to test: "))
    
    iterations = 0
    while iterations < num:
        block = getBlockNumber()
        print("current block number", block)
        next_block = block + 1
        while block < next_block:
            block = getBlockNumber()
        # get the average apr for each shard 
        print("next block reached", block, "will begin testing")
        apr_avg = getAprByShards()
        apr_avg = sorted(apr_avg.items(), key=lambda kv: kv[0])
        print("the average apr for each shard: ", apr_avg)
        iterations += 1        
    

In [7]:
R14_test()

Test-R14: Shard fairness: rate of earning on shards is similar if the block time are same
enter the number of blocks you want to test: 3
current block number 24294
next block reached 24295 will begin testing
the average apr for each shard:  [(0, 11.595348571901626), (1, 27.71951413820334), (2, 23.609600586957846), (3, 0.7216224870702569)]
current block number 24295
next block reached 24296 will begin testing
the average apr for each shard:  [(0, 11.595348571901626), (1, 27.71951413820334), (2, 23.609600586957846), (3, 0.7216224870702569)]
current block number 24296
next block reached 24297 will begin testing
the average apr for each shard:  [(0, 11.595348571901626), (1, 27.71951413820334), (2, 23.609600586957846), (3, 0.7216224870702569)]
